In [ ]:
from entsoe import EntsoePandasClient
import pandas as pd
import os

pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.max_rows", None)  # Show all rows
pd.set_option("display.width", None)  # Set width to expand to full display
pd.set_option("display.max_colwidth", None)

client = EntsoePandasClient(api_key=os.getenv("ENTSOE_API_KEY"))

start = pd.Timestamp("2024-11-10T17:00", tz="UTC")
end = pd.Timestamp("2024-11-10T18:00", tz="UTC")

# df = client.query_crossborder_flows("ES", "PT", start=start, end=end)
df = client.query_generation("PT", start=start, end=end, psr_type=None)
df = df.tz_convert("UTC")

: 

In [12]:
# df = df.resample('h').mean()
df = df.xs("Actual Aggregated", axis=1, level=1)
df

,Biomass,Fossil Gas,Fossil Hard coal,Hydro Pumped Storage,Hydro Run-of-river and poundage,Hydro Water Reservoir,Other,Solar,Wind Onshore
2015-01-01 00:00:00+00:00,307.0,454.0,1757.0,554.0,967.0,705.0,43.0,0.0,513.0
2015-01-01 01:00:00+00:00,310.0,446.0,1757.0,308.0,568.0,474.0,41.0,0.0,551.0
2015-01-01 02:00:00+00:00,316.0,446.0,1756.0,0.0,392.0,401.0,41.0,0.0,597.0
2015-01-01 03:00:00+00:00,315.0,448.0,1757.0,0.0,404.0,168.0,41.0,0.0,706.0
2015-01-01 04:00:00+00:00,316.0,448.0,1758.0,0.0,410.0,152.0,40.0,0.0,721.0
2015-01-01 05:00:00+00:00,316.0,448.0,1758.0,0.0,569.0,125.0,41.0,0.0,765.0
2015-01-01 06:00:00+00:00,323.0,449.0,1757.0,0.0,632.0,249.0,41.0,0.0,726.0
2015-01-01 07:00:00+00:00,320.0,449.0,1648.0,17.0,534.0,419.0,42.0,0.0,866.0
2015-01-01 08:00:00+00:00,321.0,448.0,1423.0,22.0,754.0,384.0,41.0,51.0,845.0
2015-01-01 09:00:00+00:00,325.0,445.0,1384.0,22.0,759.0,259.0,42.0,177.0,801.0
